### Imports and definitions

In [14]:
import pandas as pd
import numpy as np
import datetime

In [15]:
def load_degiro(filename):
    '''
    Reads an export file from DeGiro account (.csv), renames and filters columns.
    Returns: dataframe
    '''
    df = pd.read_csv(filename)
    df = (df.rename(columns={'Datum': 'date', 'Čas': 'time', 'Produkt': 'product',
                  'Reference': 'reference', 'Venue': 'venue',
                  'Počet': 'pcs_change', 'Cena': 'price',
                  'Unnamed: 8': 'price_currency', 'Hodnota': 'value_EUR',
                  'Směnný kurz': 'exchange_rate',
                  'Transaction and/or third': 'transaction_costs_EUR',
                  'Celkem': 'total_EUR', 'ID objednávky': 'ID'})
         .drop(['Hodnota v domácí měně', 'Unnamed: 10', 'Unnamed: 12',
                'Unnamed: 15', 'Unnamed: 17'], axis=1))
    return df

In [16]:
def process_degiro(df):
    '''
    Formats output from load_degiro() and returns df with unified format.
    Returns: dataframe
    '''
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%d-%m-%Y %H:%M')
    df = (df.drop(['date', 'time'], axis=1)
           .set_index('datetime')
           .sort_index()
           .loc[:, ['product', 'ISIN', 'pcs_change', 'value_EUR',
                    'transaction_costs_EUR', 'total_EUR']])
    return df

In [17]:
def report(df, nISIN, drop=True):
    '''
    Generates final report from processed df (output from process_degiro())
    for specified ISIN.
    
    Input: processed dataframe, ISIN (string)
        drop=True : drop rows where all items were sold before 2021
    
    Returns: dataframe
    '''
    nISIN = '"' + nISIN + '"'
    df1 = df.query(f'ISIN == {nISIN}').sort_index()    
    df1['row_no'] = range(0, df1.shape[0])
    df1['price_per_pc'] = abs(df1['value_EUR'] / df1['pcs_change'])    
    df1['pcs_total'] = df1['pcs_change'].cumsum()
    df1['flag_sell_2021'] = (
        np.where((df1.index.year == 2021)
                 & (df1['pcs_change'] <= 0)
                 & (df1['value_EUR'] != 0),
                 'SELL 2021', ''))
    
    df1['flag_zero'] = np.where((df1['pcs_total'] == 0)
                                & (df1['value_EUR'] != 0),
                                'ALL SOLD', '')    
    
    # drop rows with all items sold before 2021:
    prev = df1.loc[(df1.index.year < 2021)
                   & (df1['pcs_total'] == 0)
                   & (df1['value_EUR'] != 0)].row_no.max()
    if drop and prev > 0:
        df1 = df1.loc[df1['row_no'] > prev]
    
    col_list = ['row_no', 'ISIN', 'product', 'pcs_change', 'pcs_total', 'price_per_pc',
                'value_EUR', 'transaction_costs_EUR', 'total_EUR',
                'flag_sell_2021', 'flag_zero']
    
    return df1.loc[:, col_list]

### Load and prepare data

In [18]:
# file from DeGiro
FILENAME = 'Transactions_2021-12-31_II'  # without extension

ddf = load_degiro(FILENAME + '.csv')
ddf = process_degiro(ddf)
ddf.head(3)

,product,ISIN,pcs_change,value_EUR,transaction_costs_EUR,total_EUR
datetime,,,,,,
2016-11-18 09:00:00,CEZ AS,CZ0005112300,2.0,-31.23,-0.77,-32.00
2017-01-10 12:51:00,MONETA MONEY BANK AS,CZ0008040318,250.0,-758.96,-1.50,-760.46
2017-01-17 15:30:00,BANK OF AMERICA CORPOR,US0605051046,25.0,-529.00,-0.59,-529.60


### Records with NaN transaction costs (head)

In [19]:
null_df = (ddf[(ddf.transaction_costs_EUR.isna())]
           .sort_values(['datetime', 'ISIN'], ascending=True))
null_df.head()

,product,ISIN,pcs_change,value_EUR,transaction_costs_EUR,total_EUR
datetime,,,,,,
2017-10-05 00:00:00,SIBANYE GOLD LIMITED A - NON TRADEABLE,US8257242060,1.0,0.00,NaN,0.00
2017-10-16 08:58:00,SIBANYE GOLD LIMITED A - NON TRADEABLE,US8257242060,-1.0,0.00,NaN,0.00
2017-10-16 08:58:00,SIBANYE GOLD LIMITED A,US8257242060,1.0,0.00,NaN,0.00
2018-01-30 09:05:00,ISHARES S&P 500,IE0031442068,20.0,-458.60,NaN,-458.60
2018-02-23 16:11:00,ISHARES S&P 500,IE0031442068,5.0,-110.15,NaN,-110.15


### Records with 0 EUR transactions (head)

In [20]:
zero_EUR_df = ddf[ddf.value_EUR == 0].sort_values(['datetime', 'ISIN'], ascending=True)
zero_EUR_df.head()

,product,ISIN,pcs_change,value_EUR,transaction_costs_EUR,total_EUR
datetime,,,,,,
2017-07-31 15:33:00,AMAZON.COM INC. - COM - CFD,US0231351067,-2.0,0.0,-0.51,-0.51
2017-07-31 15:37:00,AMAZON.COM INC. - COM - CFD,US0231351067,2.0,0.0,-0.51,-0.51
2017-07-31 15:59:00,AMAZON.COM INC. - COM - CFD,US0231351067,-2.0,0.0,-0.51,-0.51
2017-07-31 16:12:00,AMAZON.COM INC. - COM - CFD,US0231351067,2.0,0.0,-0.51,-0.51
2017-09-15 16:00:00,AMAZON.COM INC. - COM - CFD,US0231351067,-2.0,0.0,-0.51,-0.51


### All sells in 2021

In [21]:
c1 = ddf.index.year == 2021
c2 = ddf.pcs_change < 0
all_sells_df = ddf[c1 & c2].sort_values(['datetime', 'ISIN'])

### All titles subject to sell in 2021

In [22]:
titles_df = (all_sells_df[['product','ISIN']]
             .drop_duplicates(subset='ISIN')
             .set_index('ISIN')
             .sort_values('product'))

### Volume sold (26 CZK/EUR)

In [23]:
vol = round(all_sells_df.loc[:, 'value_EUR'].sum()*26)
print(f'Volume sold in 2021: {vol} CZK')

Volume sold in 2021: 416099 CZK


### Export data to Excel file

In [24]:
with pd.ExcelWriter(FILENAME + '_output.xlsx') as writer:
    # list of all sells
    all_sells_df.to_excel(writer, sheet_name='ALL_SELLS_2021')
    
    # list of all titles sold
    titles_df.to_excel(writer, sheet_name='ALL_TITLES')
    
    # all reports in one excel sheet
    df_all = report(ddf, '')  # empty df
    for i in titles_df.index.values:
        df_all = df_all.append(report(ddf, i))
    df_all.to_excel(writer, sheet_name='ONE_SHEET')
    
    # separate sheet for each title
    for i in titles_df.index.values:
        report(ddf, i).to_excel(writer, sheet_name=i)

### Show all data

In [25]:
#for i in titles_df.index.values:
#    display(report(ddf, i))

### Show data - Moneta Money Bank

In [26]:
display(report(ddf, 'CZ0008040318'))

,row_no,ISIN,product,pcs_change,pcs_total,price_per_pc,value_EUR,transaction_costs_EUR,total_EUR,flag_sell_2021,flag_zero
datetime,,,,,,,,,,,
2017-04-12 11:19:00,9,CZ0008040318,MONETA MONEY BANK AS,750.0,750.0,3.036880,-2277.66,-3.03,-2280.69,,
2017-07-03 09:00:00,10,CZ0008040318,MONETA MONEY BANK AS,-175.0,575.0,2.962514,518.44,-1.28,517.15,,
2017-07-03 09:00:00,11,CZ0008040318,MONETA MONEY BANK AS,-300.0,275.0,2.962500,888.75,-0.89,887.86,,
2017-07-03 09:00:00,12,CZ0008040318,MONETA MONEY BANK AS,-50.0,225.0,2.962600,148.13,-0.15,147.98,,
2017-07-03 09:00:00,13,CZ0008040318,MONETA MONEY BANK AS,-25.0,200.0,2.962400,74.06,-0.07,73.99,,
2018-03-13 10:09:00,14,CZ0008040318,MONETA MONEY BANK AS,1200.0,1400.0,3.310442,-3972.53,-4.76,-3977.28,,
2018-11-29 16:25:00,15,CZ0008040318,MONETA MONEY BANK AS,-300.0,1100.0,3.080000,924.00,-1.86,922.14,,
2019-07-26 11:06:00,16,CZ0008040318,MONETA MONEY BANK AS,-33.0,1067.0,3.104848,102.46,-0.90,101.56,,
2019-07-26 11:26:00,17,CZ0008040318,MONETA MONEY BANK AS,-467.0,600.0,3.104797,1449.94,-1.71,1448.23,,
